# Coding decision tree

In decision trees, the key is to find an efficient way to split the data based on its features. There are two popular ways to do that:

1) Entropy change. The criterion for entropy is **Shannon's entropy** formula: $\displaystyle -\sum_{n=1}^{N}p_n log_{2}(p_n)$, where $N$ is the number of all possible states the system could be found in, and $p_n$ is the probability of finding the system in n-th state. For example, in binary classification tasks, the system could be found in only 2 possible states - when target variable is equal to 0 or 1. If we have [1,1,0,0,1,0,1,0,1,1,0,1], the $p_n$ for n=0 is $\frac{5}{12}$, and for n=1 is $\frac{7}{12}$.

Entropy change is called an **information gain**. $IG = E_0 - (-\sum_{n=1}^{N}p_n log_{2}(p_n))$, where $E_0$ is the initial entropy (before current split).

Higher entropy, as we know from physics course termodynamics, correlates with higher degree of chaos. Since we want to differentiate between classes, we need to minimize the entropy, or maximize the infrmation gain.

2) **Gini index**, also known as Gini impurity. It has the same variables but different formula: $G = 1 - \sum_{n=1}^{N}p_n^2$, where $p_n$ is again the probability of finding system in n-th state given $N$ is the number of all possible system states.

$\displaystyle \Delta Gini = Gini_{parent} - (Gini_{left}\frac{n_{left}}{n_{right} + n_{left}} + Gini_{right}\frac{n_{right}}{n_{right} + n_{left}})$

The formula above shows that $\displaystyle \Delta Gini$ is calculated using *weighted* gini indexes.

### Best split

To find the best split among numeric columns, *means of neighbors* are used.

$[1,2,5,6,10] -> [1.5,3.5,5.5,8]$

It helps to iterate over every possible split.

Pseudo code:

    for feature in features:
        for mean in [means of neighbors]:
            find gini impurity
            find gini gain
            
    return (best_split, best_feature) 


### Tree results

The main takeaways of the algorithm will be displayed in `print_tree()` function.

In [1]:
import pandas as pd
import numpy as np
from collections import Counter


class Node:
    """
    Node class for recursive tree growth.
    """
    def __init__(
        self,
        X: pd.DataFrame,
        y: list,
        min_samples_split=None,
        max_depth=None,
        depth=None,
        node_type=None,
        rule=None
    ):
        self.X = X
        self.y = y
        
        self.min_samples_split = min_samples_split if min_samples_split else 1
        self.max_depth = max_depth if max_depth else 5
        
        self.depth = depth if depth else 0
        
        self.features = list(self.X.columns)
        
        self.node_type = node_type if node_type else 'root'
        
        self.rule = rule if rule else ""
        
        # Calculating the counts of y in the node
        self.counts = Counter(self.y)
        
        self.GINI_impurity = self.get_gini()
        
        # Get the most frequent class
        f = None
        counts_sorted = list(sorted(self.counts.items(), key=lambda item: item[1]))
        
        if len(counts_sorted) > 0:
            f = counts_sorted[-1][0]
            
        self.f = f
        
        # Number of samples in the split
        self.n = len(y)
        
        # Initiate right and left nodes as empty nodes
        self.right = None
        self.left = None
        
        # Initiate default values for splits
        self.best_value = None
        self.best_feature = None
    
    @staticmethod
    def gini_impurity(y1_count: int, y2_count: int) -> float:
        """
        Calculates the the gini impurity given the observations of binary class count.
        """
        if type(y1_count) != int:
            y1_count = 0
        
        if type(y2_count) != int:
            y2_count = 0
        
        # Total observations
        n = y1_count + y2_count
        
        # 0 observations dataset is completely pure
        if n == 0:
            return 0.0
        
        p1 = y1_count/n
        p2 = y2_count/n
        
        g = 1 - (p1**2 + p2**2)
        
        return g
    
    @staticmethod
    def ma(x: np.array, window: int) -> np.array:
        """
        Returns the moving average (means of neighbors) of the given list.
        """
        return np.convolve(x, np.ones(window), 'valid')/window
    
    def get_gini(self):
        """
        Calculates gini impurity of the node.
        """
        y1_count, y2_count = self.counts.get(0,0), self.counts.get(1,0)  # (key, default)
        return self.gini_impurity(y1_count, y2_count)
    
    def best_split(self) -> tuple:
        """
        Calculates best split for the current node given X features and y targets.
        """
        # Create dataset for making splits
        df = self.X.copy()
        df['y'] = self.y
        
        # Initial gini index
        gini_base = self.get_gini()
        
        max_gain = 0
        
        best_value = None
        best_feature = None
        
        for feature in self.features:
            Xdf = df.dropna().sort_values(feature)
            
            # Finding moving average for sorted unique values -> values used for splits
            xmeans = self.ma(Xdf[feature].unique(), 2)
            
            for value in xmeans:
                left_count = Counter(Xdf[Xdf[feature]<value]['y'])
                right_count = Counter(Xdf[Xdf[feature]>=value]['y'])
                
                # Getting the y distribution from the Counter object
                y0_left, y1_left = left_count.get(0,0), left_count.get(1,0)
                y0_right, y1_right = right_count.get(0,0), right_count.get(1,0)
                
                gini_left = self.gini_impurity(y0_left, y1_left)
                gini_right = self.gini_impurity(y0_right, y1_right)
                
                n_left = y0_left + y1_left
                n_right = y0_right + y1_right
                
                # Calculate the weights for subnodes
                w_left = n_left/(n_left + n_right)
                w_right = n_right/(n_left + n_right)
                
                # Calculate weighted gini impurity
                gini_weighted = w_left*gini_left + w_right*gini_right
                
                gini_gain = gini_base - gini_weighted
                
                # check if this is the best split so far
                if gini_gain > max_gain:
                    best_feature = feature
                    best_value = value
                    
                    max_gain = gini_gain
                
            return (best_feature, best_value)
        
    def grow_tree(self):
        """
        Recursive method to create decision tree.
        """
        # Make df
        df = self.X.copy()
        df['y'] = self.y

        if (self.depth < self.max_depth) and (self.n >= self.min_samples_split):

            best_feature, best_value = self.best_split()

            if best_feature is not None:
                self.best_feature = best_feature
                self.best_value = best_value

                # Getting the left and right nodes
                left_df = df[df[self.best_feature] < self.best_value].copy()
                right_df = df[df[self.best_feature] >= self.best_value].copy()

                # Creating the left and the right nodes
                left = Node(
                    X=left_df[self.features],
                    y=left_df['y'].values.tolist(),
                    depth=self.depth+1,
                    max_depth=self.max_depth,
                    min_samples_split=self.min_samples_split,
                    node_type='left_node',
                    rule=f"{self.best_feature} < {round(self.best_value, 3)}"
                )

                self.left = left
                self.left.grow_tree()

                right = Node(
                    X=right_df[self.features],
                    y=right_df['y'].values.tolist(),
                    depth=self.depth+1,
                    max_depth=self.max_depth,
                    min_samples_split=self.min_samples_split,
                    node_type='right_node',
                    rule=f"{self.best_feature} >= {round(self.best_value, 3)} "
                )

                self.right = right
                self.right.grow_tree()

    def print_info(self, width=4):
        """
        Method to print the information about the tree.
        """
        const = int(self.depth * width ** 1.5)
        spaces = const * "-"

        if self.node_type == "root":
            print("Root")
        else:
            print(f"|{spaces} Split rule: {self.rule}")
        print(f"{' ' * const}   | Gini impurity of the node: {round(self.GINI_impurity, 2)}")
        print(f"{' ' * const}   | Class distribution in the node: {dict(self.counts)}")
        print(f"{' ' * const}   | Predicted class: {self.f}")

    def print_tree(self):
        """
        Prints the whole tree from top to the bottom.
        """

        self.print_info()

        if self.left is not None:
            self.left.print_tree()

        if self.right is not None:
            self.right.print_tree()

    def predict(self, X: pd.DataFrame):
        """
        Batch prediction method.
        """

        predictions = []

        for _, x in X.iterrows():  # iterates over the DataFrame as (index, Series) pair
            values = {}
            for feature in self.features:
                values.update({feature: x[feature]})

            predictions.append(self.predict_obs(values))

        return predictions

    def predict_obs(self, values: dict) -> int:
        """
        Method to predict a class given a set of features.
        """

        current_node = self
        while current_node.depth < current_node.max_depth:
            # Going from top to bottom
            best_feature = current_node.best_feature
            best_value = current_node.best_value

            if current_node.n < current_node.min_samples_split:
                break
            
            if (best_feature == None) or (best_value == None):
                break
            if (values.get(best_feature) < best_value):
                if self.left is not None:
                    current_node = current_node.left
            else:
                if self.right is not None:
                    current_node = current_node.right

        return current_node.f
    

def main():
    df = pd.read_csv('../../Kaggle/heart-disease/heart-disease.csv')
    X = df.dropna().drop(["target"], axis=1).copy()
    y = df['target'].values.tolist()
    
    clf = Node(X, y)
    clf.grow_tree()
    
    clf.print_tree()

    
if __name__ == '__main__':
    main()

Root
   | Gini impurity of the node: 0.5
   | Class distribution in the node: {1: 165, 0: 138}
   | Predicted class: 1
|-------- Split rule: age < 54.5
           | Gini impurity of the node: 0.42
           | Class distribution in the node: {1: 100, 0: 44}
           | Predicted class: 1
|---------------- Split rule: age < 42.5
                   | Gini impurity of the node: 0.34
                   | Class distribution in the node: {1: 29, 0: 8}
                   | Predicted class: 1
|------------------------ Split rule: age < 40.5
                           | Gini impurity of the node: 0.43
                           | Class distribution in the node: {1: 13, 0: 6}
                           | Predicted class: 1
|-------------------------------- Split rule: age < 39.5
                                   | Gini impurity of the node: 0.38
                                   | Class distribution in the node: {1: 12, 0: 4}
                                   | Predicted class: 1
|----------